In [8]:
from zget_data import get_responses, get_qdetails, get_questions, page_map, quest_map, attention
from collections import defaultdict
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.by import By

In [9]:
password_response, time_response, survey_response, dids = get_responses()

Total: 163; 
Attentive: 131; Non Attentive: 32


In [10]:
password = pd.DataFrame(password_response)
passwords = password['password2'].to_list()

In [11]:
driver = webdriver.Chrome()
driver.get("https://passwordmonster.com")

pass_strength = {}
for pss in passwords:
    element = driver.find_element(By.ID, 'lgd_out_pg_pass')
    element.send_keys(pss)
    element.clear()
    complexity = driver.find_element(By.ID, 'complexity-span')
    pass_strength[pss] = complexity.get_attribute("innerHTML")

In [13]:
with open("pstrength.json", "w") as fp:
    json.dump(pass_strength, fp)